In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gps-dataset/blackmesa.geojson
/kaggle/input/gps-dataset/Redv1.geojson
/kaggle/input/gps-dataset/Redv2.geojson


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gps-dataset/blackmesa.geojson
/kaggle/input/gps-dataset/Redv1.geojson
/kaggle/input/gps-dataset/Redv2.geojson


In [3]:
!pip install movingpandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.2/77.2 kB 2.8 MB/s eta 0:00:00


In [5]:
!pip install hvplot

In [6]:
# !pip install holoviews==1.18.0

In [7]:
# !pip install bokeh==3.3.1


In [8]:
# !conda install -c conda-forge holoviews=1.18.0 bokeh=3.3.1


In [9]:
import numpy as np
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import shapely as shp
import hvplot.pandas 
import matplotlib.pyplot as plt

from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts, dim
from os.path import exists
from urllib.request import urlretrieve

import warnings
warnings.filterwarnings('ignore')

plot_defaults = {'linewidth':5, 'capstyle':'round', 'figsize':(9,3), 'legend':True}
opts.defaults(opts.Overlay(active_tools=['wheel_zoom'], frame_width=300, frame_height=500))
hvplot_defaults = {'tiles':None, 'cmap':'Viridis', 'colorbar':True}

mpd.show_versions()


MovingPandas 0.17.2

SYSTEM INFO
-----------
python     : 3.10.13 | packaged by conda-forge | (main, Dec 23 2023, 15:36:39) [GCC 12.3.0]
executable : /opt/conda/bin/python3.10
machine    : Linux-5.15.133+-x86_64-with-glibc2.31

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : 3.11.1
GEOS lib   : /opt/conda/lib/libgeos_c.so
GDAL       : 3.6.4
GDAL data dir: /opt/conda/lib/python3.10/site-packages/fiona/gdal_data
PROJ       : 9.3.1
PROJ data dir: /opt/conda/share/proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 0.14.2
pandas     : 2.2.0
fiona      : 1.9.5
numpy      : 1.26.3
shapely    : 1.8.5.post1
rtree      : 1.2.0
pyproj     : 3.6.1
matplotlib : 3.7.4
mapclassify: 2.6.1
geopy      : 2.4.1
holoviews  : 1.18.1
hvplot     : 0.9.2
geoviews   : 1.11.0
stonesoup  : None


In [10]:
df = gpd.read_file('/kaggle/input/gps-dataset/Redv1.geojson')

In [11]:
df

,OBJECTID,GMT_Time,Latitude,Longitude,Altitude,Duration,Temperatur,Voltage,DOP,Satellites,...,Status,ODBA_15m,Std_O_15,FMR_4fac,FMG_4fac,FMT_4fac,FM_R,FM_G,FM_T,geometry
0,1,None,34.808915,-108.275778,1839.38,2,32.5,0,1.4,6,...,4445.58,321.270492,61.180781,0.055723,0.547232,0.364526,0.080860,0.488673,0.811327,POINT (749199.161 3855236.140)
1,2,None,34.814689,-108.284635,1832.68,35,30.0,0,1.8,6,...,4264.39,322.057377,40.621673,0.221823,0.420752,0.252407,0.164410,0.683623,0.616377,POINT (748371.425 3855854.709)
2,3,None,34.816612,-108.275953,1829.10,1,25.5,0,3.2,5,...,4299.68,299.745902,51.494526,0.072717,0.532951,0.319329,0.090142,0.510332,0.789668,POINT (749159.976 3856089.585)
3,4,None,34.817109,-108.283824,1842.23,3,24.0,0,1.6,5,...,4205.66,300.196721,50.921443,0.083579,0.534477,0.314396,0.097717,0.528006,0.771994,POINT (748438.346 3856125.196)
4,5,None,34.815958,-108.281429,1839.44,32,25.0,0,1.4,5,...,4548.24,278.168033,40.495820,0.093015,0.381977,0.286998,0.093015,0.517035,0.782965,POINT (748660.918 3856003.435)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3214,3215,None,34.666048,-108.379399,2840.65,1,22.5,0,2.0,5,...,2311.37,344.536885,43.306421,0.563474,0.578780,0.168423,0.285825,0.966925,0.333075,POINT (740131.286 3839134.990)
3215,3216,None,34.668654,-108.383121,2820.32,2,22.5,0,2.6,5,...,2240.37,354.991803,85.584536,0.270739,0.635650,0.351490,0.256930,0.899504,0.400496,POINT (739782.641 3839415.134)
3216,3217,None,34.673302,-108.382225,2731.71,2,22.5,0,3.6,5,...,2134.39,389.270492,127.894920,0.178415,0.300000,0.583698,0.240936,0.862184,0.437816,POINT (739851.385 3839932.956)
3217,3218,None,34.671203,-108.377569,2769.68,8,23.0,0,6.6,4,...,2020.69,363.668033,37.373566,0.799057,0.437491,0.079803,0.299471,0.998766,0.301234,POINT (740284.108 3839711.197)


In [12]:
df['t'] = pd.to_datetime(df['time'])
df = df.set_index('t').tz_localize(None)

In [13]:
df.head()

,OBJECTID,GMT_Time,Latitude,Longitude,Altitude,Duration,Temperatur,Voltage,DOP,Satellites,...,Status,ODBA_15m,Std_O_15,FMR_4fac,FMG_4fac,FMT_4fac,FM_R,FM_G,FM_T,geometry
t,,,,,,,,,,,,,,,,,,,,,
2021-09-07 12:45:00,1,None,34.808915,-108.275778,1839.38,2,32.5,0,1.4,6,...,4445.58,321.270492,61.180781,0.055723,0.547232,0.364526,0.080860,0.488673,0.811327,POINT (749199.161 3855236.140)
2021-09-16 13:00:00,2,None,34.814689,-108.284635,1832.68,35,30.0,0,1.8,6,...,4264.39,322.057377,40.621673,0.221823,0.420752,0.252407,0.164410,0.683623,0.616377,POINT (748371.425 3855854.709)
2021-09-19 14:15:00,3,None,34.816612,-108.275953,1829.10,1,25.5,0,3.2,5,...,4299.68,299.745902,51.494526,0.072717,0.532951,0.319329,0.090142,0.510332,0.789668,POINT (749159.976 3856089.585)
2021-09-21 12:30:00,4,None,34.817109,-108.283824,1842.23,3,24.0,0,1.6,5,...,4205.66,300.196721,50.921443,0.083579,0.534477,0.314396,0.097717,0.528006,0.771994,POINT (748438.346 3856125.196)
2021-09-21 12:45:00,5,None,34.815958,-108.281429,1839.44,32,25.0,0,1.4,5,...,4548.24,278.168033,40.495820,0.093015,0.381977,0.286998,0.093015,0.517035,0.782965,POINT (748660.918 3856003.435)


In [14]:
original_crs = df.crs
original_crs

<Projected CRS: EPSG:26912>
Name: NAD83 / UTM zone 12N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: North America - between 114°W and 108°W - onshore and offshore. Canada - Alberta; Northwest Territories; Nunavut; Saskatchewan. United States (USA) - Arizona; Colorado; Idaho; Montana; New Mexico; Utah; Wyoming.
- bounds: (-114.0, 31.33, -108.0, 84.0)
Coordinate Operation:
- name: UTM zone 12N
- method: Transverse Mercator
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [ ]:
!pip install geoviews holoviews cartopy

In [15]:
import hvplot.pandas

In [16]:
df = df.to_crs(epsg=4326)

In [17]:
one=df.hvplot(title='Geographic extent of the dataset', geo=True, tiles='OSM')

In [18]:
one

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]

In [ ]:
%load_ext holoviews.extension
%matplotlib inline

In [19]:
df = df.to_crs({'init': 'epsg:4326'})
( df.hvplot(title='OSM showing paths and fences', size=2, geo=True, tiles='OSM') +
  df.hvplot(title='Imagery showing land cover details', size=2, color='red', geo=True, tiles='EsriImagery') )

:Layout
   .Overlay.I  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.II :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]

In [20]:
from pyproj import CRS
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts, dim, Layout
from os.path import exists
from urllib.request import urlretrieve

In [21]:
temp = df.to_crs(CRS(25832))
temp['geometry'] = temp['geometry'].buffer(5)
total_area = temp.dissolve(by='No').area

In [22]:
print("The dataset covers the time between {} and {}.".format(df.index.min(), df.index.max()))

The dataset covers the time between 2021-08-10 08:45:00 and 2021-10-08 09:46:00.


In [23]:
print("That's {}".format(df.index.max() - df.index.min()))

That's 59 days 01:01:00


In [24]:
df['No'].resample('1d').count().hvplot(title='Number of records per day')

:Curve   [t]   (No)

In [25]:
df['Y-M'] = df.index.to_period('M')    
Layout([df[df['Y-M']==i].hvplot(title=str(i), size=2, geo=True, tiles='OSM') for i in df['Y-M'].unique()])

:Layout
   .Overlay.I   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.II  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.III :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]

In [26]:
t = df.reset_index().t
df = df.assign(delta_t=t.diff().values)
df['delta_t'] = df['delta_t'].dt.total_seconds()/60
pd.DataFrame(df).hvplot.hist('delta_t', title='Histogram of intervals between consecutive records (in minutes)', bins=60, bin_range=(0, 60))

:Histogram   [delta_t]   (delta_t_count)

In [27]:
df['CollarId']=5

In [29]:
tc = mpd.TrajectoryCollection(df, 'CollarId')
traj = tc.trajectories[0]
traj.add_speed(overwrite=True)
max_speed = traj.df.speed.max()
print("The highest computed speed is {:,.2f} m/s ({:,.2f} km/h)".format(max_speed, max_speed*3600/1000))

The highest computed speed is 5.82 m/s (20.96 km/h)


In [31]:
tc = mpd.TrajectoryCollection(df, 'CollarId')
traj = tc.trajectories[0]

In [32]:
pd.DataFrame(traj.df).hvplot.hist('speed', title='Histogram of speeds (in meters per minute)', bins=90)

:Histogram   [speed]   (speed_count)

In [33]:
traj.add_direction(overwrite=True)
pd.DataFrame(traj.df).hvplot.hist('direction', title='Histogram of directions', bins=90)

:Histogram   [direction]   (direction_count)

In [34]:
pd.DataFrame(traj.df).hvplot.heatmap(title='Mean speed by hour of day and month of year', 
                                     x='t.hour', y='t.month', C='speed', reduce_function=np.mean)

:HeatMap   [t.hour,t.month]   (speed)

In [35]:
traj.df['n'] = 5
pd.DataFrame(traj.df).hvplot.heatmap(title='Record count by temperature and month of year', 
                                     x='Temperatur', y='t.month', C='n', reduce_function=np.sum)

:HeatMap   [Temperatur,t.month]   (n)

In [36]:
pd.DataFrame(traj.df).hvplot.heatmap(title='Mean speed by temperature and month of year', 
                                     x='Temperatur', y='t.month', C='speed', reduce_function=np.mean)

:HeatMap   [Temperatur,t.month]   (speed)

In [37]:
traj.df['dir_class'] = ((traj.df['direction']-22.5)/45).round(0)
temp = traj.df
Layout([temp[temp['dir_class']==i].hvplot(geo=True, tiles='OSM', size=2, width=300, height=300, title=str(int(i*45))+"°") for i in sorted(temp['dir_class'].unique())])

:Layout
   .Overlay.I    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.II   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.III  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.IV   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.V    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VI   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VII  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VIII :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]

In [38]:
traj.df['speed_class'] = (traj.df['speed']*2).round(1)
temp = traj.df
plots = []
for i in sorted(temp['speed_class'].unique()):
    filtered = temp[temp['speed_class']==i]
    if len(filtered) < 20: 
        continue
    plots.append(filtered.hvplot(geo=True, tiles='EsriImagery', color='red', size=2, width=300, height=300, title=str(i/2)))
Layout(plots)

:Layout
   .Overlay.I     :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.II    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.III   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.IV    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.V     :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VI    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VII   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VIII  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.IX    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.X     :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XI    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XII   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XIII  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XIV   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XV    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XVI   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XVII  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XVIII :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XIX   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XX    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]

In [39]:
daily = mpd.TemporalSplitter(tc).split(mode='day')

In [40]:
Layout([daily.trajectories[i].hvplot(title=daily.trajectories[i].id, c='speed', line_width=2, cmap='RdYlBu') for i in range(0,7)])

:Layout
   .Overlay.I   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.II  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.III :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.IV  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.V   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.VI  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.VII :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)

In [41]:
daily_starts = daily.get_start_locations()
daily_starts.set_index(pd.to_datetime(daily_starts['time']), inplace=True) 
daily_starts['month'] = daily_starts.index.month
daily_starts.hvplot(c='month', geo=True, tiles='EsriImagery', cmap='autumn')

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (month)

In [42]:
moving = mpd.TrajectoryCollection(traj.df[traj.df['speed'] > 1.5], 'CollarId')
moving = mpd.ObservationGapSplitter(moving).split(gap=timedelta(minutes=70))

In [43]:
moving.get_start_locations().hvplot(c='month', geo=True, tiles='EsriImagery', color='red')

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]

In [44]:
daily_lengths = [traj.get_length() for traj in daily]
daily_t = [traj.get_start_time() for traj in daily]

In [45]:
daily_lengths = pd.DataFrame(daily_lengths, index=daily_t, columns=['length'])
daily_lengths.hvplot(title='Daily trajectory length')

:Curve   [index]   (length)